# Problema de Negócio (Versão Avançada)

A "Fazenda VerdeVida" planeja o mix de culturas entre Soja (duas variedades) e Milho Safrinha, maximizando lucro com restrições agronômicas, climáticas e logísticas.

**Restrições Agronômicas:**
- Limites de Potássio (K) e Fósforo (P) no solo.
- Área cultivável limitada por compactação do solo.

**Restrições Logísticas:**
- Horas de operação de máquinas (tratores, colheitadeiras).
- Capacidade finita de armazenamento nos silos.

Objetivo: Maximizar lucro total respeitando essas restrições.

# Modelagem e Coleta de Dados (Expandida)

Simulação de dados para 1500 talhões com 3 culturas: Soja Resistente, Soja Produtiva e Milho Safrinha.

Parâmetros estimados:
- Demanda de nutrientes (Potássio, Fósforo) por hectare.
- Horas de máquina necessárias por hectare.
- Produtividade para cálculo de armazenagem.

In [ ]:
!pip install pulp pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import pulp
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value, LpStatus

# --- Geração de Dados Mockados (Simulando 1500 Talhões com 3 culturas) ---
np.random.seed(42)
n_dados = 1500

dados = {
    'id_talhao': range(1, n_dados + 1),
    'cultura': np.random.choice(['Soja_Resistente', 'Soja_Produtiva', 'Milho_Safrinha'], n_dados),
}
df = pd.DataFrame(dados)

# --- Definindo parâmetros de simulação para cada cultura ---
# Soja Resistente
sr_params = {'prod': (3.5, 0.5), 'custo': (1800, 150), 'agua': (450, 50), 'k': (80, 10), 'p': (70, 8), 'horas': (10, 1.5)}
# Soja Produtiva
sp_params = {'prod': (4.8, 0.6), 'custo': (2500, 200), 'agua': (600, 60), 'k': (100, 12), 'p': (90, 10), 'horas': (12, 1.8)}
# Milho Safrinha (geralmente menos produtivo que a soja, mas com diferentes demandas)
ms_params = {'prod': (5.5, 0.7), 'custo': (2800, 250), 'agua': (700, 70), 'k': (120, 15), 'p': (100, 12), 'horas': (15, 2.0)}

def generate_data(params, size):
    return {
        'produtividade_ton_ha': np.random.normal(loc=params['prod'][0], scale=params['prod'][1], size=size),
        'custo_ha': np.random.normal(loc=params['custo'][0], scale=params['custo'][1], size=size),
        'uso_agua_m3_ha': np.random.normal(loc=params['agua'][0], scale=params['agua'][1], size=size),
        'demanda_k_kg_ha': np.random.normal(loc=params['k'][0], scale=params['k'][1], size=size), # Potássio
        'demanda_p_kg_ha': np.random.normal(loc=params['p'][0], scale=params['p'][1], size=size), # Fósforo
        'horas_maquina_ha': np.random.normal(loc=params['horas'][0], scale=params['horas'][1], size=size), # Logística
    }

mask_sr = df['cultura'] == 'Soja_Resistente'
mask_sp = df['cultura'] == 'Soja_Produtiva'
mask_ms = df['cultura'] == 'Milho_Safrinha'

for col, values in generate_data(sr_params, df[mask_sr].shape[0]).items(): df.loc[mask_sr, col] = values
for col, values in generate_data(sp_params, df[mask_sp].shape[0]).items(): df.loc[mask_sp, col] = values
for col, values in generate_data(ms_params, df[mask_ms].shape[0]).items(): df.loc[mask_ms, col] = values

# --- Cálculo das Médias para o Modelo de Otimização ---
params = df.groupby('cultura').mean()

# Preços de venda
preco_soja_ton = 2200
preco_milho_ton = 1300 # Milho tem preço diferente

# Cálculo do Lucro por Hectare
params['lucro_ha'] = 0
params.loc['Soja_Resistente', 'lucro_ha'] = (params.loc['Soja_Resistente', 'produtividade_ton_ha'] * preco_soja_ton) - params.loc['Soja_Resistente', 'custo_ha']
params.loc['Soja_Produtiva', 'lucro_ha'] = (params.loc['Soja_Produtiva', 'produtividade_ton_ha'] * preco_soja_ton) - params.loc['Soja_Produtiva', 'custo_ha']
params.loc['Milho_Safrinha', 'lucro_ha'] = (params.loc['Milho_Safrinha', 'produtividade_ton_ha'] * preco_milho_ton) - params.loc['Milho_Safrinha', 'custo_ha']

print("Parâmetros médios calculados para o novo modelo (Soja e Milho):")
print(params)

In [ ]:
# --- Recursos Disponíveis na Fazenda (Nossas Restrições) ---
# Básicas
AREA_TOTAL_DISPONIVEL_HA = 500
ORCAMENTO_TOTAL_DISPONIVEL = 1_100_000
AGUA_TOTAL_DISPONIVEL_M3 = 250_000
# Agronômicas
POTASSIO_DISPONIVEL_KG = 45000  # Limite de Potássio no solo
FOSFORO_DISPONIVEL_KG = 42000   # Limite de Fósforo no solo
AREA_NAO_COMPACTADA_HA = 400    # Limite de área sem compactação
# Logísticas
HORAS_MAQUINA_DISPONIVEIS = 6000
CAPACIDADE_SILO_TON = 2500

# --- Extraindo parâmetros para facilitar a leitura ---
# Lucro
l_sr = params.loc['Soja_Resistente', 'lucro_ha']
l_sp = params.loc['Soja_Produtiva', 'lucro_ha']
l_ms = params.loc['Milho_Safrinha', 'lucro_ha']
# Custo
c_sr = params.loc['Soja_Resistente', 'custo_ha']
c_sp = params.loc['Soja_Produtiva', 'custo_ha']
c_ms = params.loc['Milho_Safrinha', 'custo_ha']
# Água
a_sr = params.loc['Soja_Resistente', 'uso_agua_m3_ha']
a_sp = params.loc['Soja_Produtiva', 'uso_agua_m3_ha']
a_ms = params.loc['Milho_Safrinha', 'uso_agua_m3_ha']
# Nutrientes
k_sr, p_sr = params.loc['Soja_Resistente', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
k_sp, p_sp = params.loc['Soja_Produtiva', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
k_ms, p_ms = params.loc['Milho_Safrinha', ['demanda_k_kg_ha', 'demanda_p_kg_ha']]
# Logística
h_sr = params.loc['Soja_Resistente', 'horas_maquina_ha']
h_sp = params.loc['Soja_Produtiva', 'horas_maquina_ha']
h_ms = params.loc['Milho_Safrinha', 'horas_maquina_ha']
# Produtividade
prod_sr = params.loc['Soja_Resistente', 'produtividade_ton_ha']
prod_sp = params.loc['Soja_Produtiva', 'produtividade_ton_ha']
prod_ms = params.loc['Milho_Safrinha', 'produtividade_ton_ha']


# --- Modelagem Matemática Detalhada ---
print("--- 3. Modelagem Matemática (Tradução Formal Avançada) ---")
print("\nA.1. Variáveis de Decisão:")
print("   x_sr: Hectares de 'Soja_Resistente'")
print("   x_sp: Hectares de 'Soja_Produtiva'")
print("   x_ms: Hectares de 'Milho_Safrinha'")

print("\nA.2. Função Objetivo (Maximizar Lucro):")
print(f"   Maximizar Z = {l_sr:.2f}*x_sr + {l_sp:.2f}*x_sp + {l_ms:.2f}*x_ms")

print("\nA.3. Restrições:")
print(f"   a) Área Total:      x_sr + x_sp + x_ms <= {AREA_TOTAL_DISPONIVEL_HA}")
print(f"   b) Área Compac.:    x_sr + x_sp + x_ms <= {AREA_NAO_COMPACTADA_HA}  (Assumindo que todas são sensíveis)")
print(f"   c) Orçamento:       {c_sr:.0f}*x_sr + {c_sp:.0f}*x_sp + {c_ms:.0f}*x_ms <= {ORCAMENTO_TOTAL_DISPONIVEL}")
print(f"   d) Água:            {a_sr:.0f}*x_sr + {a_sp:.0f}*x_sp + {a_ms:.0f}*x_ms <= {AGUA_TOTAL_DISPONIVEL_M3}")
print(f"   e) Potássio (K):    {k_sr:.0f}*x_sr + {k_sp:.0f}*x_sp + {k_ms:.0f}*x_ms <= {POTASSIO_DISPONIVEL_KG}")
print(f"   f) Fósforo (P):     {p_sr:.0f}*x_sr + {p_sp:.0f}*x_sp + {p_ms:.0f}*x_ms <= {FOSFORO_DISPONIVEL_KG}")
print(f"   g) Horas-Máquina:   {h_sr:.0f}*x_sr + {h_sp:.0f}*x_sp + {h_ms:.0f}*x_ms <= {HORAS_MAQUINA_DISPONIVEIS}")
print(f"   h) Armazenagem:     {prod_sr:.1f}*x_sr + {prod_sp:.1f}*x_sp + {prod_ms:.1f}*x_ms <= {CAPACIDADE_SILO_TON}")

In [ ]:
# --- B.5. Implementar e Rodar o Modelo ---
modelo = LpProblem(name="Otimizacao_Mix_Culturas", sense=LpMaximize)

# Variáveis
x_sr = LpVariable(name="Hectares_Soja_Resistente", lowBound=0, cat='Continuous')
x_sp = LpVariable(name="Hectares_Soja_Produtiva", lowBound=0, cat='Continuous')
x_ms = LpVariable(name="Hectares_Milho_Safrinha", lowBound=0, cat='Continuous')

# Função Objetivo
modelo += lpSum([l_sr*x_sr, l_sp*x_sp, l_ms*x_ms]), "Lucro_Total"

# Restrições
modelo += (x_sr + x_sp + x_ms <= AREA_TOTAL_DISPONIVEL_HA, "Restricao_Area_Total")
modelo += (x_sr + x_sp + x_ms <= AREA_NAO_COMPACTADA_HA, "Restricao_Area_Nao_Compactada")
modelo += (c_sr*x_sr + c_sp*x_sp + c_ms*x_ms <= ORCAMENTO_TOTAL_DISPONIVEL, "Restricao_Orcamento")
modelo += (a_sr*x_sr + a_sp*x_sp + a_ms*x_ms <= AGUA_TOTAL_DISPONIVEL_M3, "Restricao_Agua")
modelo += (k_sr*x_sr + k_sp*x_sp + k_ms*x_ms <= POTASSIO_DISPONIVEL_KG, "Restricao_Potassio")
modelo += (p_sr*x_sr + p_sp*x_sp + p_ms*x_ms <= FOSFORO_DISPONIVEL_KG, "Restricao_Fosforo")
modelo += (h_sr*x_sr + h_sp*x_sp + h_ms*x_ms <= HORAS_MAQUINA_DISPONIVEIS, "Restricao_Horas_Maquina")
modelo += (prod_sr*x_sr + prod_sp*x_sp + prod_ms*x_ms <= CAPACIDADE_SILO_TON, "Restricao_Armazenagem")

# Resolver
modelo.solve()

# --- B.6. Obter a Solução Ótima ---
print("\n" + "="*50)
print("--- 4. Solução Computacional (Avançada) ---")
status = LpStatus[modelo.status]
print(f"Status da Solução: {status}")

if status == 'Optimal':
    print(f"\nPlano de Plantio Ótimo:")
    print(f"  - Plantar {x_sr.varValue:.2f} ha de Soja Resistente.")
    print(f"  - Plantar {x_sp.varValue:.2f} ha de Soja Produtiva.")
    print(f"  - Plantar {x_ms.varValue:.2f} ha de Milho Safrinha.")
    print(f"\nLucro Máximo Esperado: R$ {value(modelo.objective):,.2f}")
else:
    print("Não foi possível encontrar uma solução ótima. Verifique as restrições, pois elas podem ser conflitantes.")

In [ ]:
# --- Análise de Sensibilidade ---
print("\n" + "="*50)
print("--- 5. Análise de Sensibilidade (Avançada) ---")

if status == 'Optimal':
    print("\nA. Preço Sombra (Shadow Price) dos Recursos:")
    print("   Isso nos diz qual é o gargalo REAL da nossa operação!\n")
    
    for nome, restricao in modelo.constraints.items():
        preco_sombra = restricao.pi
        print(f"   - Recurso '{nome}': Preço Sombra = R$ {preco_sombra:.2f}")

        if preco_sombra > 0:
            print(f"     -> GARGALO! Cada unidade a mais deste recurso aumentaria o lucro em R$ {preco_sombra:.2f}.")
            print(f"        Decisão: Vale a pena investir para obter mais deste recurso (ex: alugar mais máquinas, comprar fertilizante, etc.)")
        else:
            print(f"     -> NÃO É GARGALO. Este recurso não está sendo 100% utilizado. Não adianta investir aqui agora.")
else:
    print("Análise de sensibilidade não aplicável.")